In [128]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import wget

In [101]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
page

<Response [200]>

## Parse the url

In [102]:
soup = BeautifulSoup(page.text,'html.parser')

In [121]:
#print(soup.prettify())

## Find the html table from the parser

In [120]:
table=soup.find_all('table')[0]

In [105]:
df = pd.DataFrame(columns=range(2))

## Update the table with row and column value 

In [106]:
row_index=0
for row in table.find_all('tr'):
    col_index=0
    columns = row.find_all('td')
    for col in columns:
        df.loc[row_index,col_index] = col.get_text().replace('\n','')
        col_index += 1
    row_index += 1

In [107]:
df.reset_index(drop=True)
df.columns=['PostalCode','Borough','Neighborhood']

## Remove Borough with "Not assigned"

In [108]:
df.drop(df[df['Borough'] == 'Not assigned'].index,axis=0,inplace=True)
df=df.reset_index(drop=True)

## Replace / with ,

In [109]:
df['Neighborhood']=df['Neighborhood'].str.replace('/',',')

In [111]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


## Checking for nan value in Neighborhood column

In [117]:
df.isnull().any()

PostalCode      False
Borough         False
Neighborhood    False
dtype: bool

In [119]:
df.shape

(103, 3)

## Load the coordinates file

In [130]:
wget.download('http://cocl.us/Geospatial_data','/Users/skdb/Desktop/Projects/Coursera_Capstone')

'/Users/skdb/Desktop/Projects/Coursera_Capstone/Geospatial_Coordinates.csv'

In [134]:
coordinates_df = pd.read_csv('Geospatial_Coordinates.csv')
coordinates_df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
coordinates_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [137]:
df.columns

Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')

## Merge two dataframe

In [140]:
toronto_merge = df
toronto_merge = toronto_merge.merge(coordinates_df,on='PostalCode')
toronto_merge.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
